In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
conda install -c pytorch pytorch torchvision

In [6]:
import os

class_names = []
labels = []
all_inputs = []

acceptable_formats = ['jpg', 'JPG', 'jpeg', 'JPEG']
root_dir = r"C:\Users\Mega-PC\Downloads\YOGA\YOGA\content\cleaned\DATASET\TRAIN"
for idx, name in enumerate(os.listdir(root_dir)):
    class_names.append(name)
    
    for root, subdir,files in os.walk(os.path.join(root_dir, name)):
        for file in files:
            if file.split('.')[1] in acceptable_formats:
                all_inputs.append(os.path.join(root_dir, name, file))
                labels.append(idx)
            else: 
#                 print(f"file {file} not included.")
                continue

all_inputs[0], labels[0], len(all_inputs), len(labels)

('C:\\Users\\Mega-PC\\Downloads\\YOGA\\YOGA\\content\\cleaned\\DATASET\\TRAIN\\downdog\\00000128.jpg',
 0,
 1001,
 1001)

In [7]:
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True

import torchvision.transforms as transform

In [8]:
my_transform = transform.Compose([
    transform.Resize((227,227)),
    transform.ToTensor()
])

In [9]:
class_names = []

In [10]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=5):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [11]:
from torch.utils.data import Dataset, DataLoader
class yogaDataset(Dataset):
    def __init__(self, root_dir, transform=my_transform):

        self.labels = []
        self.all_inputs = []
        self.transform = transform
        
        self.total_inputs = 0
        for idx, name in enumerate(os.listdir(root_dir)):
            class_names.append(name)

            for root, subdir,files in os.walk(os.path.join(root_dir, name)):
                print(f"Total files in {name}: {len(files)}")
                self.total_inputs += len(files)
                for file in files:
                    if file.split('.')[1] in acceptable_formats:
                        current_file = os.path.join(root_dir, name, file)
                        
                        #checking for dimensions
                        img = Image.open(current_file).convert('RGB')
                        
#                         print('current file : ', current_file, " : ")
                        
                        try:
                            if (transform):
                                img = self.transform(img)
                                
                                if(img.size(0) == 3):
                                    self.all_inputs.append(current_file)
                                    self.labels.append(idx)
#                                     print("added")
                                else:
#                                     print(f"{img.size()} not included. Incorrect dimension!")
                                    continue
                                    
                        except Exception as e:
#                             print(e, f"not included! transform error!")
                            continue
                            
                        
                    else:
                        continue
#                         print(f"not included. Incorrect format!")
        print("Total files processed: ", len(self.all_inputs), "/", self.total_inputs)
                    
                          
    def __len__(self):
        return len(self.all_inputs)
    
    def __getitem__(self, index): 
        
#         print(f"accessed: , {all_inputs[index]}")
        
        img = Image.open(all_inputs[index])
        
        if(self.transform is not None):
            img = self.transform(img)
        
        target = torch.tensor(self.labels[index])
        return img, target

In [12]:
train_dataset = yogaDataset(r"C:\Users\Mega-PC\Downloads\YOGA\YOGA\content\cleaned\DATASET\TRAIN", transform=my_transform)
test_dataset = yogaDataset(r"C:\Users\Mega-PC\Downloads\YOGA\YOGA\content\cleaned\DATASET\TEST", transform=my_transform)

Total files in downdog: 223
Total files in goddess: 178
Total files in plank: 264
Total files in tree: 160
Total files in warrior2: 250
Total files processed:  1001 / 1075
Total files in downdog: 97
Total files in goddess: 77
Total files in plank: 114
Total files in tree: 69
Total files in warrior2: 109
Total files processed:  428 / 466


In [ ]:
train_dataset

In [16]:
from PIL import Image


def get_train_valid_loader(batch_size, augment, random_seed, valid_size=0.1, shuffle=True):
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
        ])

    # convert RGBA to RGB
    def rgba_to_rgb(img):
        return img.convert('RGB')

    
    train_dataset.transform.transforms.insert(0, transforms.Lambda(rgba_to_rgb))

    test_dataset.transform.transforms.insert(0, transforms.Lambda(rgba_to_rgb))

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = None
    valid_loader = None

    if len(train_idx) > 0:
        train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=batch_size, sampler=train_sampler)

    if len(valid_idx) > 0:
        valid_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=batch_size, sampler=valid_sampler)

    return train_loader, valid_loader
def get_test_loader( batch_size, shuffle=True):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transform
    transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        normalize,
    ])

    dataset = test_dataset
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle
    )

    return data_loader




train_loader, valid_loader = get_train_valid_loader(batch_size = 1,
                       augment = False,                             		     random_seed = 1)

test_loader = get_test_loader(
                              batch_size = 1)



In [22]:
num_classes = 5
num_epochs = 3
batch_size = 1
learning_rate = 0.005

model = AlexNet(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  




In [ ]:
len(valid_loader.dataset)

In [20]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labs) in enumerate(train_loader): 
        if i >= len(valid_loader.dataset):
            break
        # Move tensors to the configured device
        images = images.to(device)
        labs = labs.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labs)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    

Epoch [1/20], Step [429/901], Loss: 1.5573
Epoch [2/20], Step [429/901], Loss: 1.3375


KeyboardInterrupt: 

In [21]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))   

Accuracy of the network on the 10000 test images: 24.065420560747665 %


In [ ]:
print(len(all_inputs))
len(labels)

In [18]:
len(labels)

1001

In [14]:
class_names=['downdog',
 'goddess',
 'plank',
 'tree',
 'warrior2']